In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
con.enable_load_extension(True)

In [4]:
con.load_extension("C:/Users/eugen/Desktop/Jupytier_Notebook/school-ds/sqlean-win-x64/stats.dll")

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [5]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [7]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [8]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [9]:
tasks+=1
pd.read_sql(
    '''
    SELECT COUNT(CRIM) AS CRIM, COUNT(ZN) AS ZN, COUNT(INDUS) AS INDUS, COUNT(CHAS) AS CHAS, COUNT(NOX) AS NOX
    FROM boston
    WHERE CRIM IS NULL OR ZN IS NULL OR INDUS IS NULL OR CHAS IS NULL OR NOX IS NULL
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [10]:
tasks+=1
pd.read_sql(
    '''
    SELECT COUNT(DISTINCT Uniq_CRIM) as CRIM, COUNT(DISTINCT Uniq_ZN) as ZN, 
    COUNT(DISTINCT Uniq_INDUS) as INDUS, COUNT(DISTINCT Uniq_CHAS) as CHAS, COUNT(DISTINCT Uniq_NOX) as NOX
    FROM (
    SELECT CRIM, ZN, INDUS, CHAS, NOX,
    rank() over(order by CRIM) as Uniq_CRIM,
    rank() over(order by ZN) as Uniq_ZN,
    rank() over(order by INDUS) as Uniq_INDUS,
    rank() over(order by CHAS) as Uniq_CHAS,
    rank() over(order by NOX) as Uniq_NOX
    FROM boston)
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [11]:
tasks+=1
pd.read_sql(
    '''
    SELECT median(CRIM) AS median_CRIM, median(ZN) AS median_ZN, 
    median(INDUS) AS median_INDUS, median(CHAS) AS median_CHAS, median(NOX) AS median_NOX
    FROM boston
    ''',
    con,
)

,median_CRIM,median_ZN,median_INDUS,median_CHAS,median_NOX
0,0.25651,0.0,9.69,0.0,0.538


In [12]:
pd.read_sql(
    '''
    SELECT ZN, CHAS
    FROM boston
    ''',
    con,
)

,ZN,CHAS
0,18.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
501,0.0,0.0
502,0.0,0.0
503,0.0,0.0
504,0.0,0.0


По столбцу ZN можно сделать вывод о том, что выделено очень мало участков свыше 25000 кв. м. под жилую застройку. По столбцу CHAS можно сделать вывод о том, что практически нет участков, граничащих с рекой Чарльз.

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [13]:
tasks+=1
pd.read_sql(
    '''
    SELECT (
    (SELECT avg(RM) as mean_rm_max_cost
    FROM (SELECT MEDV, RM, rank() over(order by MEDV desc) as avg_max_cost
    FROM boston)
    WHERE avg_max_cost = 1) -
    (SELECT avg(RM) as avg_25_min_cost
    FROM (SELECT MEDV, RM
    FROM boston 
    order by MEDV limit 25))) as Difference
    UNION ALL
    SELECT (
    (SELECT avg(RM) as mean_rm_max_cost
    FROM (SELECT MEDV, RM, rank() over(order by MEDV desc) as avg_max_cost
    FROM boston)
    WHERE avg_max_cost = 1) -
    (SELECT avg(RM) as avg_50_min_cost
    FROM (SELECT MEDV, RM
    FROM boston 
    order by MEDV limit 50)))
    UNION ALL
    SELECT (
    (SELECT avg(RM) as mean_rm_max_cost
    FROM (SELECT MEDV, RM, rank() over(order by MEDV desc) as avg_max_cost
    FROM boston)
    WHERE avg_max_cost = 1) -
    (SELECT avg(RM) as avg_100_min_cost
    FROM (SELECT MEDV, RM
    FROM boston 
    order by MEDV limit 100)))
    UNION ALL
    SELECT (
    (SELECT avg(RM) as mean_rm_max_cost
    FROM (SELECT MEDV, RM, rank() over(order by MEDV desc) as avg_max_cost
    FROM boston)
    WHERE avg_max_cost = 1) -
    (SELECT avg(RM) as avg_200_min_cost
    FROM (SELECT MEDV, RM
    FROM boston 
    order by MEDV limit 200)))
    UNION ALL
    SELECT (
    (SELECT avg(RM) as mean_rm_max_cost
    FROM (SELECT MEDV, RM, rank() over(order by MEDV desc) as avg_max_cost
    FROM boston)
    WHERE avg_max_cost = 1) -
    (SELECT avg(RM) as avg_300_min_cost
    FROM (SELECT MEDV, RM
    FROM boston 
    order by MEDV limit 300)))
    ''',
    con,
)

,Difference
0,1.736160
1,1.730760
2,1.596880
3,1.572295
4,1.511773


Исходя из полученных значений видно, что с увеличением выборки в сторону более дорогих домов, разница между средним количеством комнат уменьшается. Это говорит о том, что цена влияет на количество комнат. Чем она выше, тем выше среднее RM.

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [14]:
tasks+=1
pd.read_sql(
    '''
    SELECT LSTAT, rank_lstat
    FROM (SELECT MEDV, LSTAT, rank() over (order by LSTAT) as rank_lstat, rank() over (order by MEDV desc) as rank_MEDV
    FROM boston
    order by MEDV desc) tab
    WHERE rank_MEDV = 1
    ''',
    con,
)

,LSTAT,rank_lstat
0,1.73,1
1,1.92,2
2,2.88,6
3,2.96,8
4,2.97,9
5,3.16,15
6,3.26,17
7,3.32,18
8,3.70,26
9,3.73,27


Из вывовдов можно сказать, что процент населения с более низким статусом не слишком высок в домах с самой дорогой стоимостью, но есть и исключения.

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [15]:
tasks+=1
pd.read_sql(
    '''
    SELECT DISTINCT CHAS, mean_medv
    FROM (SELECT CHAS, avg(MEDV) over(PARTITION BY CHAS) AS mean_medv
    FROM boston) tab
    ORDER BY CHAS DESC
    ''',
    con,
)

,CHAS,mean_medv
0,1.0,28.440000
1,0.0,22.093843


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [16]:
tasks+=1
pd.read_sql(
    '''
    SELECT INDUS, NOX
    FROM (SELECT CRIM, ZN, INDUS, NOX, CHAS, avg(CRIM) over(PARTITION BY CHAS) as avg_CRIM,  
    avg(ZN) over(PARTITION BY CHAS) as avg_ZN, avg(INDUS) over(PARTITION BY CHAS) as avg_INDUS, 
    avg(NOX) over(PARTITION BY CHAS) as avg_NOX
    FROM boston
    ORDER BY CHAS DESC) tab
    ''',
    con,
)

,INDUS,NOX
0,19.58,0.871
1,19.58,0.871
2,19.58,0.871
3,19.58,0.871
4,19.58,0.605
...,...,...
501,11.93,0.573
502,11.93,0.573
503,11.93,0.573
504,11.93,0.573


Среднее значение выше, когда дом граничит с рекой в столбцах INDUS и NOX. INDUS показывает долю акров неторгового бизнеса на город, что говорит о том, что на границах с рекой повышенная концентрация неторгового бизнеса. Из-за повышенной концентрации неторгового бизнеса - промышленной застройки около реки, наблюдается повышенная концентрация оксидов азота в воздухе, что говорит о повышенных выбросах в атмосферу вредных веществ. 

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [17]:
tasks+=1
pd.read_sql(
    '''
    SELECT percentile(INDUS, 10) AS INDUS_10, percentile(NOX, 10) AS NOX_10, 
    percentile(INDUS, 20) AS INDUS_20, percentile(NOX, 20) AS NOX_20,
    percentile(INDUS, 50) AS INDUS_50, percentile(NOX, 50) AS NOX_50,
    percentile(INDUS, 75) AS INDUS_75, percentile(NOX, 75) AS NOX_75,
    percentile(INDUS, 100) AS INDUS_100, percentile(NOX, 100) AS NOX_100
    FROM boston
    ''',
    con,
)

,INDUS_10,NOX_10,INDUS_20,NOX_20,INDUS_50,NOX_50,INDUS_75,NOX_75,INDUS_100,NOX_100
0,2.91,0.427,4.39,0.442,9.69,0.538,18.1,0.624,27.74,0.871


Здесь также видна прямая связь пагубного влияния промышленности на окружающую среду. Чем больше промышленной застройки, тем выше концентрация оксида азота в воздухе.

In [18]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
